In [31]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd
import matplotlib.pyplot as plt # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os, shutil
for dirname, _, filenames in os.walk('kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

from fastai.vision.widgets import *
from fastai.data.all import *
from fastai.vision.core import *
from fastbook import *


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

nbdev 1.1.4 requires fastcore>=1.2.0, but you'll have fastcore 1.0.20 which is incompatible.


In [7]:
#Directory Paths
flower_path = '/kaggle/input/104-flowers-garden-of-eden/jpeg-512x512'
TRAIN_DIR  = flower_path + '/train/'
VAL_DIR  = flower_path + '/val'
TEST_DIR  = flower_path + '/test/'
New_TRAIN_DIR = flower_path + '/train1/'

In [9]:
flowers = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
dls = flowers.dataloaders(TRAIN_DIR, batch_size=32)

In [ ]:
dls.train.show_batch(max_n=4, nrows=1)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)


In [ ]:
correct = 0
total = len(learn.dls.valid_ds)
correct = (interp.targs == interp.preds.argmax(axis = 1)).sum()
accuracy = correct.float()/total

In [ ]:
accuracy

In [ ]:
interp.plot_top_losses(6, nrows=6)

In [ ]:
#Base model achieved a benchmark of 78.2%. Next we try Augmentations along with Cross Validation set.

In [24]:
flowers = flowers.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = flowers.dataloaders(TRAIN_DIR, batch_size = 32)

In [25]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(10)

epoch,train_loss,valid_loss,error_rate,time
0,2.199491,1.290951,0.325882,01:34


epoch,train_loss,valid_loss,error_rate,time
0,1.295039,0.852193,0.215686,01:37
1,0.994688,0.714506,0.184314,01:38
2,0.847436,0.655450,0.170588,01:40
3,0.665726,0.557967,0.142353,01:36
4,0.528955,0.523753,0.132941,01:39
5,0.387406,0.493375,0.121176,01:39
6,0.271018,0.491756,0.118431,01:37
7,0.188944,0.480624,0.115686,01:36
8,0.162426,0.463423,0.113333,01:35
9,0.144095,0.456875,0.110588,01:36


In [26]:
interp = ClassificationInterpretation.from_learner(learn)


In [27]:
correct = 0
total = len(learn.dls.valid_ds)
correct = (interp.targs == interp.preds.argmax(axis = 1)).sum()
accuracy = correct.float()/total

In [28]:
accuracy

tensor(0.8894)

In [15]:
#Data Augmentation gave us significant improvement in accuracy from 78 to 89.37%

In [47]:
test_ids = []
predictions = []
for dirname, _, filenames in os.walk(TEST_DIR):
    for filename in filenames:
          k = plt.imread(os.path.join(dirname,filename))
          pred = learn.predict(k)[1]
          predictions.append(pred)
          test_ids.append(filename.split(".")[0])

In [50]:
# Write the submission file
np.savetxt(
    '/kaggle/working/submission.csv',
    np.rec.fromarrays([test_ids, predictions]),
    fmt=['%s', '%d'],
    delimiter=',',
    header='id,label',
    comments='',
)